![NYC Skyline](nyc.jpg)

Welcome to New York City, one of the most-visited cities in the world. There are many Airbnb listings in New York City to meet the high demand for temporary lodging for travelers, which can be anywhere between a few nights to many months. In this project, we will take a closer look at the New York Airbnb market by combining data from multiple file types like `.csv`, `.tsv`, and `.xlsx`.

Recall that **CSV**, **TSV**, and **Excel** files are three common formats for storing data. 
Three files containing data on 2019 Airbnb listings are available to you:

**data/airbnb_price.csv**
This is a CSV file containing data on Airbnb listing prices and locations.
- **`listing_id`**: unique identifier of listing
- **`price`**: nightly listing price in USD
- **`nbhood_full`**: name of borough and neighborhood where listing is located

**data/airbnb_room_type.xlsx**
This is an Excel file containing data on Airbnb listing descriptions and room types.
- **`listing_id`**: unique identifier of listing
- **`description`**: listing description
- **`room_type`**: Airbnb has three types of rooms: shared rooms, private rooms, and entire homes/apartments

**data/airbnb_last_review.tsv**
This is a TSV file containing data on Airbnb host names and review dates.
- **`listing_id`**: unique identifier of listing
- **`host_name`**: name of listing host
- **`last_review`**: date when the listing was last reviewed

In [89]:
# Import necessary packages
import pandas as pd
import numpy as np
from functools import reduce

# Begin coding here ...
# Use as many cells as you like

In [90]:
price_df = pd.read_csv('data/airbnb_price.csv')
room_df = pd.read_excel('data/airbnb_room_type.xlsx')
review_df = pd.read_csv('data/airbnb_last_review.tsv', sep = '\t')

In [91]:
print(price_df.head())
print(room_df.head())
print(review_df.head())

   listing_id        price                nbhood_full
0        2595  225 dollars         Manhattan, Midtown
1        3831   89 dollars     Brooklyn, Clinton Hill
2        5099  200 dollars     Manhattan, Murray Hill
3        5178   79 dollars  Manhattan, Hell's Kitchen
4        5238  150 dollars       Manhattan, Chinatown
   listing_id                                description        room_type
0        2595                      Skylit Midtown Castle  Entire home/apt
1        3831            Cozy Entire Floor of Brownstone  Entire home/apt
2        5099  Large Cozy 1 BR Apartment In Midtown East  Entire home/apt
3        5178            Large Furnished Room Near B'way     private room
4        5238         Cute & Cozy Lower East Side 1 bdrm  Entire home/apt
   listing_id    host_name   last_review
0        2595     Jennifer   May 21 2019
1        3831  LisaRoxanne  July 05 2019
2        5099        Chris  June 22 2019
3        5178     Shunichi  June 24 2019
4        5238          Ben 

In [92]:
dfs = [price_df, room_df, review_df]
df = reduce(lambda left, right: pd.merge(left, right, on = 'listing_id', how='outer'), dfs)
print(df.head())


   listing_id        price  ...    host_name   last_review
0        2595  225 dollars  ...     Jennifer   May 21 2019
1        3831   89 dollars  ...  LisaRoxanne  July 05 2019
2        5099  200 dollars  ...        Chris  June 22 2019
3        5178   79 dollars  ...     Shunichi  June 24 2019
4        5238  150 dollars  ...          Ben  June 09 2019

[5 rows x 7 columns]


In [93]:
df['last_review'] = pd.to_datetime(df['last_review'], infer_datetime_format= True)
df['last_review'] = df['last_review'].dt.strftime('%Y-%m-%d')
df_sorted = df.sort_values(by = 'last_review', axis = 0, ascending= True)
print(df_sorted)
earliest_review = df_sorted.iloc[0,6]
most_recent_review = df_sorted.iloc[len(df)-1,6]
print(earliest_review)
print(most_recent_review)

       listing_id        price  ...      host_name last_review
12007    21751907   40 dollars  ...  Aura Angelica  2019-01-01
10430    19708955  700 dollars  ...          Karen  2019-01-01
14845    25931882   65 dollars  ...            Can  2019-01-01
7203     13936825   75 dollars  ...        Shlomit  2019-01-01
17123    28813184  125 dollars  ...        Carolyn  2019-01-01
...           ...          ...  ...            ...         ...
23952    34933891  120 dollars  ...        Jessica  2019-07-08
9595     18331097  100 dollars  ...   Jimmy &Cindy  2019-07-08
9505     18173787   48 dollars  ...       Danielle  2019-07-08
17563    29256134   88 dollars  ...          Scott  2019-07-08
58          18152  200 dollars  ...       Victoria  2019-07-09

[25209 rows x 7 columns]
2019-01-01
2019-07-09


In [94]:
df['room_type'] = df['room_type'].str.lower()
print(df.groupby('room_type').count())
private_rooms = 11356


                 listing_id  price  ...  host_name  last_review
room_type                           ...                        
entire home/apt       13266  13266  ...      13262        13266
private room          11356  11356  ...      11352        11356
shared room             587    587  ...        587          587

[3 rows x 6 columns]


In [95]:
df['price'] = df['price'].str.replace('dollars','').astype(int)
avg_listing_price = df['price'].mean().round(2)
print(avg_listing_price)

141.78


In [99]:
review_dates_dict = {'first_reviewed': earliest_review, 'last_reviewed': most_recent_review, 'nb_private_rooms': private_rooms, 'avg_price': avg_listing_price}
review_dates = pd.DataFrame(review_dates_dict, index=[0])
print(review_dates)

  first_reviewed last_reviewed  nb_private_rooms  avg_price
0     2019-01-01    2019-07-09             11356     141.78
